# epann

Evolutionary Plastic Artificial Neural Networks


**Back to [Part 3: Neuroevolution](03neuroevolution.ipynb)**

# Compositional Pattern Producing Networks

### Definition

**CPPN**s are an abstraction of inheritance and development constructed from simple artificial neural networks.

Their goal is take in a possible connection in a final agent's brain as coordinates within a defined substrate, and output some feature of that connection, such as it's weight and learning parameters.

By representing an agent this way, it is not necessary to encode every connection in the final agent phenotype when performing evolutionary computation to find good solutions.

Instead, we can indirectly encode agents with much smaller genotypes that describe the simple CPPN genome. An indirect encoding simplifies our representation of a solution, and allows us to search through a larger portion of the space of possible solutions to solve a task.

### Example

Let's start with a simple example using the **epann** package.

We will construct a population of 5 agents with very simple genome neural networks (CPPNs) to demonstrate how they are initialized, how their structure is related to the final agent brain, and how they change (mutate and reproduce) over the course of evolution.

In [3]:
from epann.core.population.population import Population

num_agents = 5

pop = Population(num_agents)

for agent in pop.genomes.keys():
    print 'Agent', agent, '-', pop.genomes[agent]

Agent 0 - <epann.core.population.genome.cppn.CPPN instance at 0x7f4d58307ab8>
Agent 1 - <epann.core.population.genome.cppn.CPPN instance at 0x7f4d58307c68>
Agent 2 - <epann.core.population.genome.cppn.CPPN instance at 0x7f4d58307d88>
Agent 3 - <epann.core.population.genome.cppn.CPPN instance at 0x7f4d58307ea8>
Agent 4 - <epann.core.population.genome.cppn.CPPN instance at 0x7f4d58307fc8>




As you can see, each agent is defined as an instance of a CPPN object. Within this object are attributes that define its genotype, which can then be used to construct a phenotype for the agent.

Let's set aside the first agent (**Agent 0**) and take a look at this genome.


In [4]:
index = 0
current_agent = pop.genomes[index]

Most importantly for our discussion, the current agent has two sets of genome lists that will be modified over the course of evolution among its attributes. 

#### The Node Genome

It has a *node genome*:

In [5]:
print current_agent.nodes

{0: {'activation': 'linear', 'type': 'input'}, 1: {'activation': 'linear', 'type': 'input'}, 2: {'activation': 'linear', 'type': 'input'}, 3: {'activation': 'linear', 'type': 'input'}, 4: {'activation': 'linear', 'type': 'input'}, 5: {'activation': 'ReLU', 'type': 'output'}}


Its *node genome* is a dictionary of genes that describe the characteristics of individual nodes in the CPPN. Each key is a node in the genome, and each nested dictionary is that particular node's attributes.

For example, **Node 5** is an output node ('type') with a unique activation function ('activation').

(**Note:** it might seem odd that an output node does not have a more traditional activation function, such as the sigmoid. Neurons in CPPNs can have a variety of activation functions that are selected for their ability to introduce repetition or symmetry, which gives rise to the network's pattern producing capabilities. More on this distinction later.)

For now, we can at least observe the possible activation functions output nodes can be assigned to:

In [7]:
from epann.core.tools.utils.activations import Activation

acts = Activation()
print acts.tags

['x_cubed', 'linear', 'sigmoid', 'ramp', 'gauss', 'abs_value', 'tan_h', 'step', 'ReLU', 'sine']




Nodes within the CPPN (except for the input nodes) can have any of these activation functions. For now, let's set the activation function to something simple. This will become clear why when we get to explaining the substrate, and we will change it back when we're done. Once you feel like you have gotten the hang of the relationship between the substrate and the CPPN genome, you can set the output node activation to any of the strings in the above list and see what the substrates look like (then just select Run All from the Cell pull down menu).

Note: some values are not set up to be sampled with more than one input value (i.e. a meshgrid), so just play around with it a bit to see which ones are working properly. (for example, use 'step', not 'ReLU')


In [8]:
# Save the old randomly generated activation function
old_output_act = current_agent.nodes[5]['activation']

# Re-assign the ouput node activation to something simple
current_agent.nodes[5]['activation'] = 'ramp'

We start a generatioin off with 5 agents that have the same number of input and output nodes. As a result, every agent will have identical node genomes when they are initialized, save the specific activation functions assigned to the output nodes. 

In [10]:
# Input nodes
print '\nInput nodes are equivalent across the population when initialized...\n'
for agent in range(num_agents):
    print '\n- Agent', agent
    for node in range(5):
        print '    Node', node, ':', pop.genomes[agent].nodes[node]
        
# Output nodes
print '\nWhile output nodes differ in their specific activation functions...\n'
for agent in range(num_agents):
    print '\n- Agent', agent
    print '    Node', 5, ':', pop.genomes[agent].nodes[5]


Input nodes are equivalent across the population when initialized...


- Agent 0
    Node 0 : {'activation': 'linear', 'type': 'input'}
    Node 1 : {'activation': 'linear', 'type': 'input'}
    Node 2 : {'activation': 'linear', 'type': 'input'}
    Node 3 : {'activation': 'linear', 'type': 'input'}
    Node 4 : {'activation': 'linear', 'type': 'input'}

- Agent 1
    Node 0 : {'activation': 'linear', 'type': 'input'}
    Node 1 : {'activation': 'linear', 'type': 'input'}
    Node 2 : {'activation': 'linear', 'type': 'input'}
    Node 3 : {'activation': 'linear', 'type': 'input'}
    Node 4 : {'activation': 'linear', 'type': 'input'}

- Agent 2
    Node 0 : {'activation': 'linear', 'type': 'input'}
    Node 1 : {'activation': 'linear', 'type': 'input'}
    Node 2 : {'activation': 'linear', 'type': 'input'}
    Node 3 : {'activation': 'linear', 'type': 'input'}
    Node 4 : {'activation': 'linear', 'type': 'input'}

- Agent 3
    Node 0 : {'activation': 'linear', 'type': 'input'}
    N

#### The Connection Genome

The CPPN also has a *connection genome* that keeps track of the connections between these nodes:

In [11]:
print current_agent.connections

{0: {'enable_bit': 1, 'in_node': 5, 'weight': -0.9982661689386628, 'out_node': 0}, 1: {'enable_bit': 1, 'in_node': 5, 'weight': -1.2156983232571874, 'out_node': 1}, 2: {'enable_bit': 1, 'in_node': 5, 'weight': -0.5880236110439007, 'out_node': 2}, 3: {'enable_bit': 1, 'in_node': 5, 'weight': 0.6567556335806954, 'out_node': 3}, 4: {'enable_bit': 1, 'in_node': 5, 'weight': -0.038336371949467915, 'out_node': 4}}




Just like any neural network you are accustomed to seeing, a CPPN is composed of an input layer (**Nodes 0-4**) and an output layer (with a single output node, **Node 5**).

Each agent begins with 6 total nodes which are fully connected, making 5 initial weights. Although agents in the population are structurally identical (they have the same number of initial nodes in their CPPN), the weights of these connections will not be the same for each agent.

Let's compare a single connection across the population - **Connection 0**, between **Node 0** and **Node 5** to show this fact:


In [13]:
print 'Connection weights are randomly initialized across the population for the same connection...\n'
for agent in range(num_agents):
    print '\n- Agent', agent
    print '    Connection', 0, ':', pop.genomes[agent].connections[0]

Connection weights are randomly initialized across the population for the same connection...


- Agent 0
    Connection 0 : {'enable_bit': 1, 'in_node': 5, 'weight': -0.9982661689386628, 'out_node': 0}

- Agent 1
    Connection 0 : {'enable_bit': 1, 'in_node': 5, 'weight': 0.6299693154308184, 'out_node': 0}

- Agent 2
    Connection 0 : {'enable_bit': 1, 'in_node': 5, 'weight': -0.8320610820144777, 'out_node': 0}

- Agent 3
    Connection 0 : {'enable_bit': 1, 'in_node': 5, 'weight': 1.5970228792615537, 'out_node': 0}

- Agent 4
    Connection 0 : {'enable_bit': 1, 'in_node': 5, 'weight': 0.546065592137416, 'out_node': 0}


**Move on to [Part 5: HyperNEAT](05hyperneat.ipynb)**